파이프라인을 사용한 효율적인 워크플로

In [20]:
from IPython.display import Image

In [21]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/'
                 'machine-learning-databases'
                 '/breast-cancer-wisconsin/wdbc.data', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [22]:
from sklearn.preprocessing import LabelEncoder
X = df.loc[:, 2:].values
y = df.loc[:,1].values
le = LabelEncoder()
y = le.fit_transform(y)
le.classes_

array(['B', 'M'], dtype=object)

In [23]:
le.transform(['B','M'])

array([0, 1])

In [24]:
# 데이터 분리
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =  train_test_split(X,y,test_size=0.2, stratify=y,random_state=1)

파이프라인으로 변환기와 추정기 연결

In [25]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch06/figures/06_01.png', width=500)

In [26]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression()
)
pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
test_acc = pipe_lr.score(X_test,y_test)
test_acc

0.956140350877193

K-겹 교차 검증을 사용한 모델 성능 평가
  - 홀드아웃 방법

In [27]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch06/figures/06_02.png', width=500)

 K겹 교차검증

In [28]:
Image(url='https://raw.githubusercontent.com/rickiepark/ml-with-pytorch/main/ch06/figures/06_03.png', width=500)

In [50]:
from sklearn.model_selection import StratifiedKFold, cross_validate
import numpy as np
print(len(X_train))
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
scores = []
for k, (train,test) in enumerate(kfold):
  pipe_lr.fit(X_train[train], y_train[train])
  score = pipe_lr.score(X_train[test],y_train[test])
  scores.append(score)
  print(f"폴드 : {k+1} 클래스분포:{np.bincount(y_train[train]) } 정확도:{score}")



455
폴드 : 1 클래스분포:[256 153] 정확도:0.9347826086956522
폴드 : 2 클래스분포:[256 153] 정확도:0.9347826086956522
폴드 : 3 클래스분포:[256 153] 정확도:0.9565217391304348
폴드 : 4 클래스분포:[256 153] 정확도:0.9565217391304348
폴드 : 5 클래스분포:[256 153] 정확도:0.9347826086956522
폴드 : 6 클래스분포:[257 153] 정확도:0.9555555555555556
폴드 : 7 클래스분포:[257 153] 정확도:0.9777777777777777
폴드 : 8 클래스분포:[257 153] 정확도:0.9333333333333333
폴드 : 9 클래스분포:[257 153] 정확도:0.9555555555555556
폴드 : 10 클래스분포:[257 153] 정확도:0.9555555555555556


In [46]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=pipe_lr,X=X_train,y=y_train,cv=10,n_jobs=-1)
print(f"정확도 점수: {scores} 정확도 : {np.mean(scores)}")

정확도 점수: [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556] 정확도 : 0.9495169082125603


In [54]:
from sklearn.model_selection import cross_val_predict
preds = cross_val_predict(
    estimator=pipe_lr,
    X=X_train,
    y=y_train,cv=10,
    n_jobs=-1,
    method = 'predict_proba'
)
preds[:10], np.argmax(preds[:10],axis=1),y_train[:10]

(array([[9.93982352e-01, 6.01764759e-03],
        [7.64328337e-01, 2.35671663e-01],
        [9.72683946e-01, 2.73160539e-02],
        [8.41658121e-01, 1.58341879e-01],
        [9.97144940e-01, 2.85506043e-03],
        [9.99803660e-01, 1.96339882e-04],
        [9.99324159e-01, 6.75840609e-04],
        [2.12145074e-06, 9.99997879e-01],
        [1.28668437e-01, 8.71331563e-01],
        [7.76260670e-04, 9.99223739e-01]]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1]))